Imports

In [21]:
from pathlib import Path
import datetime
import geopandas as gpd

In [22]:
def get_shapefiles(root_dir='.'):
    """
    Recursively finds all .shp files in a directory and its subdirectories.
    
    Args:
        root_dir (str): The path to the starting directory.
        
    Returns:
        dictionary: A dictionary where Keys = filename (no ext) and Values = metadata dict.
    """
    dictionary = {}
    
    # Iterate over all .shp files
    for path in Path(root_dir).rglob('*.shp'):
        
        # Get basic file statistics (size, time)
        stats = path.stat()
        
        metadata = {
            'full_path': str(path),
            'parent_folder': path.parent.name,
            # Convert size to Kilobytes for readability
            'size_kb': round(stats.st_size / 1024, 2),
            # specific modification timestamp
            'modified_timestamp': stats.st_mtime, 
            # Human readable date
            'modified_date': datetime.datetime.fromtimestamp(stats.st_mtime).strftime('%Y-%m-%d')
        }

        # Assign to dictionary using the stem (filename without extension)
        # Note: If two files have the same name in different folders, the last one found wins.
        dictionary[path.stem] = metadata

    return dictionary



In [23]:
get_shapefiles()

{'PctAutoc_2022': {'full_path': 'data_miteco\\autoctonia_2022\\PctAutoc_2022.shp',
  'parent_folder': 'autoctonia_2022',
  'size_kb': 64156.84,
  'modified_timestamp': 1707734031.434325,
  'modified_date': '2024-02-12'},
 'CifraPob2023': {'full_path': 'data_miteco\\cifras_poblacion_2023\\CifraPob2023.shp',
  'parent_folder': 'cifras_poblacion_2023',
  'size_kb': 64156.84,
  'modified_timestamp': 1707733744.934531,
  'modified_date': '2024-02-12'},
 'DensPob2023': {'full_path': 'data_miteco\\densidad_poblacion_2023\\DensPob2023.shp',
  'parent_folder': 'densidad_poblacion_2023',
  'size_kb': 64156.84,
  'modified_timestamp': 1707733916.773321,
  'modified_date': '2024-02-12'},
 'EdadMedia2023': {'full_path': 'data_miteco\\edad_media_2023\\EdadMedia2023.shp',
  'parent_folder': 'edad_media_2023',
  'size_kb': 64156.84,
  'modified_timestamp': 1707733939.988658,
  'modified_date': '2024-02-12'},
 'NumAfi_2022': {'full_path': 'data_miteco\\num_afiliados-1000_2022\\NumAfi_2022.shp',
  'pare